In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from scipy.spatial.distance import pdist, cdist
from scipy.spatial.distance import squareform

import folium
import os
import time
from joblib import dump, load

In [1]:
class SVRRegressorRBF:
    
    def __init__(self):
        self.algorithm = 'SVR_RBF'
        
    def create_svr_grid(self, C, gamma, cv):
        '''
        Create a grid search for a SVR model with RBF kernel
        '''

        steps = [
             ('scaler', StandardScaler()),
             ('svr', SVR(kernel = 'rbf', cache_size = 500))
        ]

        pipe = Pipeline(steps = steps)

        param_grid = {'svr__C': C,
                  'svr__gamma': gamma}

        scoring = {'R2': 'r2', 'MSE': 'neg_mean_squared_error'}
        grid = GridSearchCV(estimator = pipe, param_grid = param_grid,
                            scoring = scoring, return_train_score = True,
                            refit = 'R2', cv = cv, verbose = 2)
        return pipe, grid
    
    def fit(self, X_train, y_train, C = [1, 5, 50, 100], gamma = [0.05, 0.1, 0.5, 1], cv = 5):
        '''
        Find the best parameters using grid search and fit the model using these parameters.
        '''
        # Creating pipeline and grid search
        pipe, grid = self.create_svr_grid(C, gamma, cv)
        
        # Fitting grid search
        grid.fit(X_train, y_train)
        
        # Updating pipeline parameters
        pipe.set_params(**grid.best_params_)
        
        t1 = time.time()
        pipe.fit(X_train, y_train)
        t2 = time.time()
        train_time = t2 - t1
        
        self.train_time = train_time
        self.model = pipe
        
    def predict(self, X_test):
        
        y_hat = self.model.predict(X_test)
        
        return y_hat
    
    
    def get_params(self):
        '''
        Returns a dictionary of the model parameters
        SVR RBF parameters: kernel, C, gamma
        '''
        
        params = ['kernel', 'C', 'gamma']
        param_dict = {}
        
        for param in params:
            param_dict[param] = getattr(self.model['svr'], param)
        
        return param_dict
    